<a href="https://colab.research.google.com/github/neelabhro/CLEAVE_Docker/blob/main/CLEAVE_public_container.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [1]:
import os, re
from getpass import getpass

with open('cleave-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

password = getpass('enter your expeca password:')
os.environ['OS_PASSWORD'] = password

enter your expeca password:··········


Install required packages and dependencies. Ignore the warnings.

In [2]:
!pip uninstall -q -y moviepy
!pip install -q jedi
!pip install -q git+https://github.com/KTH-EXPECA/python-chi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.8/323.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.9/225.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.4/205.4 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.6/252.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.6/512.6 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

Import packages

In [3]:
import json
from loguru import logger
import chi.network, chi.container, chi.network
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed

In the next cell, we reserve the required equipment and resources to form an end to end experiment setup. We reserve EP5G network, one Advantech router and one worker to run the workloads.

In [4]:
# worker reservation
name = "worker-05"

found = False
leaseslist = list_reservations(brief=True)
for lease_dict in leaseslist:
  if lease_dict["name"] == name+"-lease":
    worker_reservation_id = lease_dict["reservation_id"]
    found = True

if not found:
  worker_lease = reserve(
    { "type":"device", "name":name, "duration": { "days":7, "hours":0 } }
  )
  worker_reservation_id = worker_lease["reservations"][0]["id"]

print(json.dumps(leaseslist,indent=4))

2024-02-26 17:38:18.201 | INFO     | chi.expeca:reserve:167 - reserving worker-05
2024-02-26 17:38:20.692 | INFO     | chi.expeca:wait_until_lease_status:62 - waiting 120 seconds for worker-05-lease with id 1a56e054-92c1-4819-93ca-85f1d14a6180 to become "ACTIVE"
2024-02-26 17:38:25.945 | INFO     | chi.expeca:wait_until_lease_status:69 - lease worker-05-lease with id 1a56e054-92c1-4819-93ca-85f1d14a6180 is PENDING.
2024-02-26 17:38:31.211 | INFO     | chi.expeca:wait_until_lease_status:69 - lease worker-05-lease with id 1a56e054-92c1-4819-93ca-85f1d14a6180 is PENDING.
2024-02-26 17:38:36.470 | INFO     | chi.expeca:wait_until_lease_status:69 - lease worker-05-lease with id 1a56e054-92c1-4819-93ca-85f1d14a6180 is PENDING.
2024-02-26 17:38:41.744 | INFO     | chi.expeca:wait_until_lease_status:69 - lease worker-05-lease with id 1a56e054-92c1-4819-93ca-85f1d14a6180 is PENDING.
2024-02-26 17:38:47.010 | INFO     | chi.expeca:wait_until_lease_status:69 - lease worker-05-lease with id 1a56e0

[]


Now we start running the container.

In [6]:
publicnet = chi.network.get_network("serverpublic")
container_name = "my-public-node"
chi.container.create_container(
    name = container_name,
    image = "samiemostafavi/sshd-image",
    reservation_id = worker_reservation_id,
    environment = {
        "DNS_IP":"8.8.8.8",
        "GATEWAY_IP":"130.237.11.97",
        "PASS":"expeca"
    },
    mounts = [],
    nets = [
        { "network" : publicnet['id'] },
    ],
    labels = {
        "networks.1.interface":"eno3np0",
        "networks.1.ip":"130.237.11.117/27",
        "networks.1.gateway":"130.237.11.97",
        "capabilities.privileged":"true",
    },
)
chi.container.wait_for_active("my-public-node")
logger.success("created my-public-node container.")

2024-02-26 17:43:03.795 | SUCCESS  | __main__:<cell line: 24>:24 - created my-public-node container.


Now you can ssh into the container via
```
ssh root@130.237.11.115
```
From anywhere. Also, you can run a remote command here. The result will be printed on the logs page in Chameleon.

In [ ]:
command = "ip route show"
result = chi.container.execute(
    container_ref=container_name,
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

2024-02-24 17:06:10.520 | INFO     | __main__:<cell line: 6>:6 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}
